<img align="center" src="https://raw.githubusercontent.com/FUlyankin/Parsers/master/images%20/cats.jpg" height="1200" width="1200"> 

# Домашка 6: классификация

Давайте попрбобуем построить модель, предсказывающую пол пользователя

In [ ]:
import pandas as pd               # уже знакомый вам пакет для работы с таблицами
import numpy as np                # смутно знакомый вам пакет для работы с матрицами
import matplotlib.pyplot as plt   # уже смутно знакомый вам пакет для картинок :3
import matplotlib
import seaborn as sns             # ещё один пакет для картинок 
plt.style.use('ggplot')     # правильный (очень красивый) стиль у графиков

%matplotlib inline

In [ ]:
# Метрики для измерения качества алгоритмов 
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score, roc_curve, auc

In [ ]:
import itertools

# Функция для отрисовки roc-auc кривой    
def roc_auc_pic(y_hat_test, y_test):    
    fpr_test, tpr_test, thresholds_test = roc_curve(y_test, y_hat_test)
    roc_auc_test = roc_auc_score(y_test, y_hat_test)

    matplotlib.rcParams['figure.figsize'] = (8, 8)
    plt.plot(fpr_test, tpr_test, label='Test ROC AUC {0}'.format(roc_auc_test))
    plt.plot([0, 1], [0, 1], '--', color=(0.6, 0.6, 0.6))
    plt.xlim([0, 1])
    plt.ylim([0, 1])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.legend(loc='lower right')
    plt.show() 
    pass

# 1. Снова подгружаем данные

В очередной раз подгрузим немного данных. 

In [ ]:
df = pd.read_csv('../data/vk_download/vk_main.csv', sep='\t')   # подгружаем датасет
df.head()

In [ ]:
df.columns

Ровно также, как мы это делали на семинаре, предобработаем данные. Если вы не понимаете какую-то из строчек в следущем большом кодище, выясните как она работает. Напишите в чат тех-поддержки, семинаристу, сходите к гадалке. Сделайте хоть что-то, потому что скоро такой код вам придётся писать самостоятельно. 

In [ ]:
# выбрасываем из выборки все переменные, в которых слишком много пропусков
isnull = df.isnull().sum()
df = df[isnull[isnull <= 56].index]

# Выборка с переменными-счётчиками
variables_cnt = [item for item in df.columns if item[-3:] == 'cnt']

# Оставляем только информативные переменные (картинки можно посмотреть в семинарской тетрадке)
variables_cnt = [item for item in variables_cnt if len(df[item].unique()) > 10]

# логарифмируем счётчики
df[variables_cnt] = df[variables_cnt].apply(lambda w: np.log(w + 1))

# отбираем дамми - переменные в том числе наш указатель пола
variables_dummy = [item for item in df.columns if item[-5:] == 'dummy']

# средние, медианы и максимумы
variables_mean = [item for item in df.columns if item[-4:] == 'mean']
variables_median = [item for item in df.columns if item[-6:] == 'median']
variables_max = [item for item in df.columns if item[-3:] == 'max']

# все полезные переменные
useful_variables = variables_cnt + variables_dummy + variables_mean + \
                   variables_median + variables_max + ['friends_mail_from_course_pct']

# заполняем пропуски нулями
df[useful_variables] = df[useful_variables].fillna(0)
df.head()

# 2. Подготовка к моделированию.

Начнём с простого. 

__[1]__ Разделите выборку на тренировочную и тестовую. Запишите обучающую выборку в табличку `df_train`, а тестовую в `df_test`.

In [ ]:
# Ваш код

Для удобства запишем в `y` то, что мы хотим прогнозировать.

In [ ]:
y_train = df_train['male_dummy'].get_values()
y_test = df_test['male_dummy'].get_values()

Посмотрим как соотносятся доли парней и девушек в тренировочной и тестовой выборках.

In [ ]:
np.mean(y_train), np.mean(y_test)

В `X` запишем то, по чему мы будем прогнозировать. 

In [ ]:
# Забираем в выборку лучших из лучших 
X_train = df_train[useful_variables].get_values()
X_test = df_test[useful_variables].get_values()

# 3. Метод ближайшего соседа

Начнём моделирование с метода ближайшего соседа. 

__[1]__ Проскалируйте переменные. Поясните в виде текста зачем нужно это делать. Надо ли скалировать переменные для деревьев? 

In [ ]:
# Ваш код 

__[1]__ Обучите на тренировочной выборке алгоритм 10 ближайших соседей. Запишите прогнозы алгоритма в переменную `y_predict`.

In [ ]:
# Ваш код

Построим для получившегося алгоритма roc-кривую на тестовой выборке. 

In [ ]:
# Для леса
roc_auc_pic(y_predict, y_test)

Посмотрим какой получилось accuracy (доля правильных ответов). 

In [ ]:
accuracy_score(y_test, y_predict)

__[1]__ Какой вывод о прогнозной силе алгоритма вы можете сделать? Можно ли в данной ситуации в качестве метрики использовать `acuracy`? 

__Ответ:__ 

# 4. Пытаемся улучшить работу алгоритма

Давайте попытаемся выжать из метода ближайших соседей максимум. 

__[2]__ Попробуйте перебрать разное количество соседей и разные дистанции. Насколько получилось улучшить работу классификатора? 

In [ ]:
# Ваш код

Какое сочетания параметров оказалось оптимальным? Какое качество показывает алгоритм? 

In [ ]:
grid_knn.best_score_

In [ ]:
grid_knn.best_params_

# 5. Дерево 

__[2]__ Продолжаем наше путешествие в классификацию. Постройте решающее дерево. Какое качество оно показывает на тренировочной выборке? А на тестовой? Что такое переобучение? Правда ли, что дерево переобучилось? 

In [ ]:
# Ваш код

__Ответ:__ 

# 6. Случайный лес 

__[2]__ Попробуйте обучить случайный лес, посмотрите на качество его работы по метрике roc_auc. 

In [ ]:
# Ваш код 

* Зачем деревья объединяют в лес? 

__Ответ:__ 

* Какой из трёх обученных выше алгоритмов показал наилучшее качество? 

__Ответ:__ 

* Как считате, удалось ли нам научиться предсказывать, что студент первого курса поставит в паблик лайк? С чем это на ваш взгляд связано? 

__Ответ:__ 


Свои возмущение и ненависть, либо любовь и обожание можно оставить в [Анонимке по 6 домашке](https://docs.google.com/forms/d/e/1FAIpQLSeKVcEfJqBD4XpDclz6QnAGb5QInQ2843iAuSneVtbBkBaL5Q/viewform)